In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt 
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image

torch.manual_seed(1)

In [2]:
dim_x = 512 * 512
dim_h = 32
dim_out = 6
model = torch.nn.Sequential(
    torch.nn.Linear(dim_x, dim_h),
    torch.nn.ReLU(),
    torch.nn.Linear(dim_h, dim_out),
)

In [3]:
def train(model, train_data_loader, val_data_loader=None, learning_rate=0.001, num_epoch=1, plot=False):
  
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#     optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.7)

    iters, losses, train_acc, val_acc = [], [], [], []
    epoch = 0 
    iteration = 0
    while (epoch < num_epoch):
        for batch_id, (data, target) in enumerate(train_data_loader):
            #############################################
            #To Enable GPU Usage
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
            #############################################
            out = model(data)             # forward pass
            loss = criterion(out, target) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
            # Save the current model (checkpoint) to a file
#             model_path = get_model_name(model.name, learning_rate, epoch)
#             torch.save(model.state_dict(), model_path)
        losses.append(float(loss))
        iters.append(iteration)
        if epoch%5==0:
            print("Epoch: " + str(epoch) + " Loss: " + str(losses[-1]))
        if plot:
            # save the current training information
            train_acc.append(get_accuracy(train_data_loader, model=model))
            if val_data_loader is not None:
                val_acc.append(get_accuracy(val_data_loader, model=model))
        iteration += 1
        epoch += 1
#         print(epoch) 
    
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()
    if plot:
        plt.title("Training Curve")
        plt.plot(iters, train_acc, label="Train")
        if val_data_loader is not None:
            plt.plot(iters, val_acc, label="Validation")
        plt.xlabel("Iterations")
        plt.ylabel("Training Accuracy")
        plt.legend(loc='best')
        plt.show()

In [4]:
def get_accuracy(data_loader, model=None, out=None):
    
    correct = 0
    total = 0
    
    for imgs, labels in data_loader:
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
        if out is not None:
            output = out
        else:
            output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [5]:
import os
import _pickle as pickle 
import torch
import numpy as np
from multiprocessing import Pool

class Data:
    
    def __init__(self, path_to_pickle_folders, replace_classes = {}, maximum_per_folder = None):
        
        if type(path_to_pickle_folders) != list:
            path_to_pickle_folders = [path_to_pickle_folders]
        
        self.data = []
        
        for folder in path_to_pickle_folders:
            print("Unpacking", os.path.basename(folder))
            working_label = os.path.basename(folder)
            
            if os.path.basename(folder) in replace_classes:
                working_label = replace_classes[os.path.basename(folder)]
            
            files_to_unpickle = [os.path.join(folder, img) for img in os.listdir(folder)]
            files_to_unpickle = files_to_unpickle[:maximum_per_folder]
            
#             Single threaded
#             for img in files_to_unpickle:
#                 self.addPickle(img)
            
#             Multithreaded
            p = Pool()
            results = p.map(self.parsePickle, files_to_unpickle)
            p.close()
            p.join()      
            
            # add to data
            for file in results:
                self.data.append({
                    working_label : file
                })

        self.convetLabels()
        self.dataToTensor()
    
    def dataToTensor(self):
        i = 0
        for data_dict in self.data:
            array = list(data_dict.values())[0]
            array = torch.Tensor(array).unsqueeze(0)
            self.data[i] = {
                list(data_dict.keys())[0] : array
            }
            i+=1
    
    def convetLabels(self):
        all_labels = np.array([list(data.keys())[0] for data in self.data])
        unique_labels = list(np.unique(all_labels))
        self.label_dict = {label:unique_labels.index(label) for label in unique_labels}
    
    def parsePickle(self, path_to_pickle):
        try:
            f=open(path_to_pickle,'rb')
            img=pickle.load(f)
            f.close()
            return img
        except:
            pass
    
    def __getitem__(self, idx):
        ''' Return img, label'''
        data = self.data[idx]
        img = list(data.values())[0]
        word_label = list(data.keys())[0]
        label = self.label_dict[word_label]

        return img, label
    
    def __len__(self):
        return len(self.data)

In [ ]:
training_folders = [
    "Processed/train/epidural",
    "Processed/train/intraparenchymal",
    "Processed/train/subarachnoid",
    "Processed/train/intraventricular",
    "Processed/train/subdural",
    "Processed/train/nohem",
]

train_data = Data(training_folders, replace_classes = {
    "epidural":"any", 
    "intraparenchymal":"any", 
    "subarachnoid":"any", 
    "intraventricular":"any", 
    "subdural":"any", 
}, maximum_per_folder = 1000)

Unpacking epidural
Unpacking intraparenchymal


In [ ]:
train_data[-1][0].shape

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=64)